<a href="https://colab.research.google.com/github/evandwh/ST554---Spring-2026---NCSU/blob/main/Whitfield_ST554_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ST554 - HW4**

**Author** - Evan Whitfield

**Class** - ST554

**Semester** - Spring 2026


# **Summarizing Data Numerically - HW3 Code**

[HW3 Notebook](https://colab.research.google.com/drive/1_xaz74anRJZEP5k0MSUovFv1xS-nU5rY?usp=sharing)

*Code is slightly modified due from original version*

## Read in the data

In [4]:
import pandas as pd
import matplotlib.pyplot as plt


URL =  "https://www4.stat.ncsu.edu/~online/datasets/StudentData.txt"

student_data = pd.read_csv(URL, sep = ";")

student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## Summarize the data

### Categorical Variables

In this data set, I am interested in how the jobs of the parents of the student might affect their preformance/results.

In [5]:
student_data.Fjob.value_counts(dropna = False)


,count
Fjob,
other,217
services,111
teacher,29
at_home,20
health,18


In [6]:
student_data.Mjob.value_counts(dropna = False)


,count
Mjob,
other,141
services,103
at_home,59
teacher,58
health,34


Out of all of the students, 20 students had fathers that worked at home, while 59 had mothers that worked at home.

In [7]:
student_data["Fjob_cat"] = student_data.Fjob.astype("category")
student_data["Mjob_cat"] = student_data.Mjob.astype("category")
student_data["guardian_cat"] = student_data.guardian.astype("category")

student_data.Fjob_cat = student_data.Fjob_cat.cat.rename_categories(["At Home", "Health", "Other", "Services", "Teacher"])
student_data.Mjob_cat = student_data.Mjob_cat.cat.rename_categories(["At Home", "Health", "Other", "Services", "Teacher"])
student_data.guardian_cat = student_data.guardian_cat.cat.rename_categories(["Father", "Mother", "Other"])

pd.crosstab(student_data.Fjob_cat, student_data.Mjob_cat)

Mjob_cat,At Home,Health,Other,Services,Teacher
Fjob_cat,,,,,
At Home,7,0,5,6,2
Health,2,6,2,4,4
Other,33,17,104,42,21
Services,15,10,24,43,19
Teacher,2,1,6,8,12


Only 7 of the students had both a mother and a father that worked from home. Meanwhile, 12 students had a mother and a father that were both Teachers.

In [8]:
my_table = pd.crosstab(
    [student_data.guardian_cat, student_data.Fjob_cat],
    student_data.Mjob_cat,
    margins = True,
    rownames = ["Guardian","Father Job"],
    colnames = ["Mother Job"])

my_table

Mother Job           At Home  Health  Other  Services  Teacher  All
Guardian Father Job                                                
Father   At Home           2       0      3         0        0    5
         Health            0       4      2         1        0    7
         Other             3       4     20         6        1   34
         Services          5       3     11        13        3   35
         Teacher           0       0      3         4        2    9
Mother   At Home           5       0      2         4        2   13
         Health            1       2      0         3        4   10
         Other            26      11     74        34       19  164
         Services          9       7     10        25       15   66
         Teacher           2       1      3         4       10   20
Other    At Home           0       0      0         2        0    2
         Health            1       0      0         0        0    1
         Other             4       2     10         2        1   19
         Services          1       0      3         5        1   10
All                       59      34    141       103       58  395

Of the 12 students that had both a mother and a father listed as a teacher, 2 of them had their father listed as the guardian, while 10 had their mother listed as their guardian. No student had both parents as teachers, but had "Other" listed as their guardian.

Create a conditional two-way table.

I chose to create a conditional two-way table that mirrored my three-way table above, in order to check the results. Here, the father is listed as the guardian and it matches with the corresponding portion of the three-way table above.

In [9]:
pd.crosstab(
    student_data.loc[student_data["guardian_cat"] == "Father", "Fjob_cat"],
    student_data.loc[student_data["guardian_cat"] == "Father", "Mjob_cat"],
    margins = True
)

Mjob_cat,At Home,Health,Other,Services,Teacher,All
Fjob_cat,,,,,,
At Home,2,0,3,0,0,5
Health,0,4,2,1,0,7
Other,3,4,20,6,1,34
Services,5,3,11,13,3,35
Teacher,0,0,3,4,2,9
All,10,11,39,24,6,90


Just to practice something different, I decided to do a subset of the table above by looking at only the students who had the father listed in the Other or Service job category. Again, this does not refill anything new from above and was chosen to check for correctness.

In [10]:
my_table.loc[("Father", ("Other", "Services")), : ]

Mother Job           At Home  Health  Other  Services  Teacher  All
Guardian Father Job                                                
Father   Other             3       4     20         6        1   34
         Services          5       3     11        13        3   35

### Numerical Variables

#### Measures of center and Spread

I am interested in the mean and standard deviation of the three variables: "age", "absences", and "G3". I decided to run a for loop to find the summary statistics I desired for each of the three variables.

In [11]:
vars = ["age", "absences", "G3"]
for i in vars:
    print("Mean of " + i + " is " + str(student_data[i].mean()))
    print("Standard Deviaition of " + i + " is " + str(student_data[i].std()) + "\n")


Mean of age is 16.696202531645568
Standard Deviaition of age is 1.2760427246056245

Mean of absences is 5.708860759493671
Standard Deviaition of absences is 8.003095687108177

Mean of G3 is 10.415189873417722
Standard Deviaition of G3 is 4.5814426109978434



To dive deeper into the numerical summary, I was interested if the gender of the student made an impact on the three variables of interest. Here, I decided to only look at the rows were "sex" == "F".

In [12]:
for i in vars:
    print("Mean of " + i + " for females is " + str(student_data.loc[student_data["sex"] == "F", i].mean()))
    print("Standard Deviaition of " + i + " for females is " + str(student_data.loc[student_data["sex"] == "F", i].std()) +"\n")

Mean of age for females is 16.73076923076923
Standard Deviaition of age for females is 1.2016958164931697

Mean of absences for females is 6.216346153846154
Standard Deviaition of absences for females is 9.44581916132164

Mean of G3 for females is 9.966346153846153
Standard Deviaition of G3 for females is 4.622338337431135



#### Grouping by one variable.

Instead of only looking at the summary statistics of one gender, I decided to group the data by "sex" and look at the summary statistics across both genders.

I did it once using the methods for `.mean()` and `.std()`, and then one using `.pivot_table` for practicing and comparison.

In [13]:
student_data.groupby("sex")[vars].mean()

,age,absences,G3
sex,,,
F,16.730769,6.216346,9.966346
M,16.657754,5.144385,10.914439


In [14]:
student_data.groupby("sex")[vars].std()

,age,absences,G3
sex,,,
F,1.201696,9.445819,4.622338
M,1.356181,5.980749,4.495297


In [15]:
pd.pivot_table(data = student_data, values = vars, index = "sex", aggfunc = ["mean", "std"])

mean                            std                    
            G3  absences        age        G3  absences       age
sex                                                              
F     9.966346  6.216346  16.730769  4.622338  9.445819  1.201696
M    10.914439  5.144385  16.657754  4.495297  5.980749  1.356181

Here, I noticed that the variables of interest were not ordered in the same way as the `vars` list was created. There must be some sort of inherent ordering that `.pivot_table` lists the values of interest. Requires further study.

#### Grouping by two variables.
After looking at gender, I wondered if `school` made any impact on the variables of interest.

Again, I did it two different ways. First, using the `.mean()` and `.std` methods, and then using the `.pivot_table()` method.

In [16]:
student_data.groupby(["sex", "school"])[vars].mean()

age  absences         G3
sex school                                
F   GP      16.579235  6.639344   9.972678
    MS      17.840000  3.120000   9.920000
M   GP      16.457831  5.222892  11.060241
    MS      18.238095  4.523810   9.761905

In [17]:
student_data.groupby(["sex", "school"])[vars].std()

age  absences        G3
sex school                              
F   GP      1.173426  9.852259  4.604622
    MS      0.746101  4.702127  4.846992
M   GP      1.263005  6.215695  4.594725
    MS      0.995227  3.669047  3.491486

In [18]:
pd.pivot_table(data = student_data, values = vars, index = ["sex", "school"], aggfunc = ["mean", "std"])

mean                            std                    
                   G3  absences        age        G3  absences       age
sex school                                                              
F   GP       9.972678  6.639344  16.579235  4.604622  9.852259  1.173426
    MS       9.920000  3.120000  17.840000  4.846992  4.702127  0.746101
M   GP      11.060241  5.222892  16.457831  4.594725  6.215695  1.263005
    MS       9.761905  4.523810  18.238095  3.491486  3.669047  0.995227

#### Correlation

Finally, we were tasked with finding the correlation matrix relating all of the numerical variables in our data set. Probably to no ones surprise, there seems to be a high correlation between the G1, G2, and G3 variables.

In [19]:
student_data[["age", "absences", "G1", "G2", "G3"]].corr()

,age,absences,G1,G2,G3
age,1.000000,0.175230,-0.064081,-0.143474,-0.161579
absences,0.175230,1.000000,-0.031003,-0.031777,0.034247
G1,-0.064081,-0.031003,1.000000,0.852118,0.801468
G2,-0.143474,-0.031777,0.852118,1.000000,0.904868
G3,-0.161579,0.034247,0.801468,0.904868,1.000000
